# Chapter 15: SOLID Principles in Python

This notebook covers the five SOLID principles of object-oriented design and how to apply them
idiomatically in Python. SOLID principles help create code that is easier to maintain, extend, and test.

## Key Concepts
- **S**ingle Responsibility Principle: one reason to change
- **O**pen/Closed Principle: open for extension, closed for modification
- **L**iskov Substitution Principle: subtypes must be substitutable
- **I**nterface Segregation Principle: prefer small, focused interfaces
- **D**ependency Inversion Principle: depend on abstractions, not concretions
- Python-specific tools: `Protocol`, `ABC`, and duck typing

## Single Responsibility Principle (SRP)

Each class or module should have **one reason to change**. If a class handles both business logic
and persistence, changes to either concern force modifications to the same class.

In [ ]:
# BAD: This class has multiple responsibilities
class UserManagerBad:
    """Handles user logic AND persistence - two reasons to change."""

    def __init__(self) -> None:
        self._users: dict[str, dict] = {}

    def create_user(self, name: str, email: str) -> dict:
        user = {"name": name, "email": email}
        self._users[email] = user
        return user

    def save_to_file(self, path: str) -> None:
        # Persistence logic mixed in with domain logic
        pass


# GOOD: Separate responsibilities into distinct classes
class User:
    """Domain object - only responsible for user data."""

    def __init__(self, name: str, email: str) -> None:
        self.name = name
        self.email = email

    def __repr__(self) -> str:
        return f"User(name={self.name!r}, email={self.email!r})"


class UserRepository:
    """Only responsible for storing and retrieving users."""

    def __init__(self) -> None:
        self._store: dict[str, User] = {}

    def save(self, user: User) -> None:
        self._store[user.email] = user

    def find_by_email(self, email: str) -> User | None:
        return self._store.get(email)


class UserService:
    """Only responsible for user business logic."""

    def __init__(self, repo: UserRepository) -> None:
        self._repo = repo

    def register(self, name: str, email: str) -> User:
        if self._repo.find_by_email(email):
            raise ValueError(f"User {email} already exists")
        user = User(name, email)
        self._repo.save(user)
        return user


# Usage
repo = UserRepository()
service = UserService(repo)

alice = service.register("Alice", "alice@example.com")
print(f"Registered: {alice}")
print(f"Found: {repo.find_by_email('alice@example.com')}")

try:
    service.register("Alice Again", "alice@example.com")
except ValueError as e:
    print(f"Error: {e}")

## Open/Closed Principle (OCP)

Software entities should be **open for extension** but **closed for modification**.
New behavior should be added by writing new code, not by changing existing code.

In [ ]:
from abc import ABC, abstractmethod


# Base class is closed for modification
class Discount(ABC):
    """Abstract discount strategy - extend by subclassing."""

    @abstractmethod
    def apply(self, price: float) -> float: ...


# Open for extension - add new discounts without changing existing code
class NoDiscount(Discount):
    def apply(self, price: float) -> float:
        return price


class PercentageDiscount(Discount):
    def __init__(self, percent: float) -> None:
        self.percent = percent

    def apply(self, price: float) -> float:
        return price * (1 - self.percent / 100)


class FixedDiscount(Discount):
    def __init__(self, amount: float) -> None:
        self.amount = amount

    def apply(self, price: float) -> float:
        return max(0, price - self.amount)


# This function works with ANY discount - never needs modification
def calculate_total(price: float, discount: Discount) -> float:
    return round(discount.apply(price), 2)


# Demonstrate with different discounts
price = 100.0
print(f"Original price: ${price:.2f}")
print(f"No discount:    ${calculate_total(price, NoDiscount()):.2f}")
print(f"20% off:        ${calculate_total(price, PercentageDiscount(20)):.2f}")
print(f"$15 off:        ${calculate_total(price, FixedDiscount(15)):.2f}")


# Adding a new discount type requires NO changes to existing code
class BuyOneGetOneDiscount(Discount):
    def apply(self, price: float) -> float:
        return price / 2


print(f"BOGO:           ${calculate_total(price, BuyOneGetOneDiscount()):.2f}")

## Liskov Substitution Principle (LSP)

Subtypes must be **substitutable** for their base types without altering the correctness of the
program. If `S` is a subtype of `T`, then objects of type `T` can be replaced with objects of type `S`
without breaking anything.

In [ ]:
# BAD: Violating LSP - Square changes Rectangle's behavior
class Rectangle:
    def __init__(self, width: float, height: float) -> None:
        self._width = width
        self._height = height

    @property
    def width(self) -> float:
        return self._width

    @width.setter
    def width(self, value: float) -> None:
        self._width = value

    @property
    def height(self) -> float:
        return self._height

    @height.setter
    def height(self, value: float) -> None:
        self._height = value

    def area(self) -> float:
        return self._width * self._height


class SquareBad(Rectangle):
    """Violates LSP: setting width also changes height."""

    def __init__(self, side: float) -> None:
        super().__init__(side, side)

    @Rectangle.width.setter
    def width(self, value: float) -> None:
        self._width = value
        self._height = value  # Unexpected side effect!


# This function expects Rectangle behavior
def resize_and_check(shape: Rectangle) -> None:
    shape.width = 5
    shape.height = 10
    expected = 50  # 5 * 10
    actual = shape.area()
    print(f"  {type(shape).__name__}: expected={expected}, actual={actual}, "
          f"correct={expected == actual}")


print("BAD design (LSP violation):")
resize_and_check(Rectangle(1, 1))
resize_and_check(SquareBad(1))  # Breaks expectations!


# GOOD: Use separate abstractions
class Shape(ABC):
    @abstractmethod
    def area(self) -> float: ...


class RectangleGood(Shape):
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height


class SquareGood(Shape):
    def __init__(self, side: float) -> None:
        self.side = side

    def area(self) -> float:
        return self.side ** 2


print("\nGOOD design (LSP respected):")
shapes: list[Shape] = [RectangleGood(5, 10), SquareGood(7)]
for s in shapes:
    print(f"  {type(s).__name__}.area() = {s.area()}")

## Interface Segregation Principle (ISP)

Clients should not be forced to depend on interfaces they do not use.
Prefer **small, focused interfaces** over large, monolithic ones.
Python's `Protocol` class is ideal for this.

In [ ]:
from typing import Protocol


# BAD: One large interface forces all implementers to define everything
class WorkerBad(ABC):
    @abstractmethod
    def work(self) -> str: ...

    @abstractmethod
    def eat(self) -> str: ...

    @abstractmethod
    def sleep(self) -> str: ...


# A robot cannot eat or sleep - forced to provide dummy implementations
class RobotBad(WorkerBad):
    def work(self) -> str:
        return "Robot working"

    def eat(self) -> str:
        raise NotImplementedError("Robots don't eat!")  # Code smell

    def sleep(self) -> str:
        raise NotImplementedError("Robots don't sleep!")


# GOOD: Segregated interfaces using Protocol
class Workable(Protocol):
    def work(self) -> str: ...


class Feedable(Protocol):
    def eat(self) -> str: ...


class Restable(Protocol):
    def sleep(self) -> str: ...


# Each class only implements what it actually supports
class Human:
    def work(self) -> str:
        return "Human working"

    def eat(self) -> str:
        return "Human eating"

    def sleep(self) -> str:
        return "Human sleeping"


class Robot:
    def work(self) -> str:
        return "Robot working"


# Functions accept only the interface they need
def assign_task(worker: Workable) -> str:
    return worker.work()


def schedule_lunch(eater: Feedable) -> str:
    return eater.eat()


human = Human()
robot = Robot()

# Both can work
print(assign_task(human))
print(assign_task(robot))

# Only humans eat
print(schedule_lunch(human))
# schedule_lunch(robot)  # mypy would flag this as an error

## Dependency Inversion Principle (DIP)

High-level modules should not depend on low-level modules. Both should depend on **abstractions**.
Abstractions should not depend on details; details should depend on abstractions.

In [ ]:
# BAD: High-level NotificationService depends directly on low-level EmailSender
class EmailSenderDirect:
    def send(self, to: str, message: str) -> str:
        return f"Email to {to}: {message}"


class NotificationServiceBad:
    def __init__(self) -> None:
        self._sender = EmailSenderDirect()  # Tight coupling!

    def notify(self, user: str, message: str) -> str:
        return self._sender.send(user, message)


# GOOD: Depend on an abstraction (Protocol)
class MessageSender(Protocol):
    def send(self, to: str, message: str) -> str: ...


class EmailSender:
    def send(self, to: str, message: str) -> str:
        return f"Email to {to}: {message}"


class SmsSender:
    def send(self, to: str, message: str) -> str:
        return f"SMS to {to}: {message}"


class SlackSender:
    def send(self, to: str, message: str) -> str:
        return f"Slack to #{to}: {message}"


class NotificationService:
    """High-level module depends on abstraction, not concretions."""

    def __init__(self, sender: MessageSender) -> None:
        self._sender = sender  # Injected dependency

    def notify(self, user: str, message: str) -> str:
        return self._sender.send(user, message)


# Easy to swap implementations
for sender in [EmailSender(), SmsSender(), SlackSender()]:
    service = NotificationService(sender)
    print(service.notify("alice", "Your order shipped!"))

## Protocol Classes for Interface Segregation

Python's `Protocol` (from `typing`) enables **structural subtyping** - a class satisfies a
Protocol if it has the right methods, without needing to inherit from it. This is Python's
formalization of duck typing.

In [ ]:
from typing import Protocol, runtime_checkable


@runtime_checkable
class Renderable(Protocol):
    """Any object that can render itself to a string."""
    def render(self) -> str: ...


@runtime_checkable
class Serializable(Protocol):
    """Any object that can serialize to a dict."""
    def to_dict(self) -> dict: ...


# These classes don't inherit from any Protocol - they just match the shape
class HtmlWidget:
    def __init__(self, content: str) -> None:
        self.content = content

    def render(self) -> str:
        return f"<div>{self.content}</div>"

    def to_dict(self) -> dict:
        return {"type": "html", "content": self.content}


class JsonResponse:
    def __init__(self, data: dict) -> None:
        self.data = data

    def render(self) -> str:
        import json
        return json.dumps(self.data)


class PlainText:
    def __init__(self, text: str) -> None:
        self.text = text

    def render(self) -> str:
        return self.text

    def to_dict(self) -> dict:
        return {"type": "text", "content": self.text}


def display(item: Renderable) -> None:
    print(f"  Rendered: {item.render()}")


def export(item: Serializable) -> None:
    print(f"  Exported: {item.to_dict()}")


print("Renderable items:")
renderables: list[Renderable] = [HtmlWidget("Hello"), JsonResponse({"ok": True}), PlainText("Hi")]
for item in renderables:
    display(item)

print("\nSerializable items:")
serializables: list[Serializable] = [HtmlWidget("Hello"), PlainText("Hi")]
for item in serializables:
    export(item)

# runtime_checkable allows isinstance checks
print(f"\nHtmlWidget is Renderable: {isinstance(HtmlWidget('x'), Renderable)}")
print(f"HtmlWidget is Serializable: {isinstance(HtmlWidget('x'), Serializable)}")
print(f"JsonResponse is Serializable: {isinstance(JsonResponse({}), Serializable)}")

## ABC vs Protocol: When to Use Which

Python offers two approaches to defining interfaces:

| Feature | `ABC` | `Protocol` |
|---------|-------|------------|
| Typing style | Nominal (must inherit) | Structural (duck typing) |
| Inheritance required | Yes | No |
| Can provide default implementations | Yes | Yes (but unusual) |
| Runtime isinstance checks | Yes (always) | Only with `@runtime_checkable` |
| Best for | Internal hierarchies | External/third-party code |
| Enforces at class definition | Yes | No (checked by mypy) |

In [ ]:
from abc import ABC, abstractmethod
from typing import Protocol


# ABC approach: Enforced at class definition time
class AnimalABC(ABC):
    @abstractmethod
    def speak(self) -> str: ...

    def describe(self) -> str:
        """Default implementation - shared by all subclasses."""
        return f"I am a {type(self).__name__} and I say: {self.speak()}"


class Dog(AnimalABC):
    def speak(self) -> str:
        return "Woof!"


class Cat(AnimalABC):
    def speak(self) -> str:
        return "Meow!"


# ABC catches missing methods immediately
try:
    class BadAnimal(AnimalABC):
        pass  # Forgot to implement speak()
    BadAnimal()  # TypeError raised HERE
except TypeError as e:
    print(f"ABC enforcement: {e}")


# Protocol approach: No inheritance needed
class Speaker(Protocol):
    def speak(self) -> str: ...


class Parrot:
    """No inheritance from Speaker - just matches the shape."""
    def speak(self) -> str:
        return "Polly wants a cracker!"


def make_noise(animal: Speaker) -> str:
    return animal.speak()


# All three work with the Speaker protocol
print(f"\nDog: {make_noise(Dog())}")
print(f"Cat: {make_noise(Cat())}")
print(f"Parrot: {make_noise(Parrot())}")

# ABC gives shared behavior for free
print(f"\nDog describes: {Dog().describe()}")
print(f"Cat describes: {Cat().describe()}")

## Practical Example: Applying All Five Principles

Here is a small reporting system that demonstrates all five SOLID principles working together.

In [ ]:
from typing import Protocol
from dataclasses import dataclass


# ISP: Small, focused protocols
class DataSource(Protocol):
    def fetch(self) -> list[dict]: ...


class Formatter(Protocol):
    def format(self, data: list[dict]) -> str: ...


class OutputTarget(Protocol):
    def write(self, content: str) -> str: ...


# SRP: Each class has a single responsibility
class SalesDataSource:
    """Responsible only for providing sales data."""
    def fetch(self) -> list[dict]:
        return [
            {"product": "Widget", "revenue": 1000},
            {"product": "Gadget", "revenue": 2500},
            {"product": "Gizmo", "revenue": 750},
        ]


class CsvFormatter:
    """Responsible only for CSV formatting."""
    def format(self, data: list[dict]) -> str:
        if not data:
            return ""
        headers = ",".join(data[0].keys())
        rows = [headers]
        for row in data:
            rows.append(",".join(str(v) for v in row.values()))
        return "\n".join(rows)


class TextFormatter:
    """Responsible only for text table formatting."""
    def format(self, data: list[dict]) -> str:
        lines = []
        for row in data:
            lines.append(" | ".join(f"{k}: {v}" for k, v in row.items()))
        return "\n".join(lines)


class ConsoleOutput:
    """Responsible only for console output."""
    def write(self, content: str) -> str:
        return f"[Console]\n{content}"


# DIP: ReportGenerator depends on abstractions (protocols), not concrete classes
# OCP: New data sources, formatters, or outputs can be added without modifying this class
class ReportGenerator:
    """Orchestrates report generation using injected dependencies."""

    def __init__(
        self,
        source: DataSource,
        formatter: Formatter,
        output: OutputTarget,
    ) -> None:
        self._source = source
        self._formatter = formatter
        self._output = output

    def generate(self) -> str:
        data = self._source.fetch()
        formatted = self._formatter.format(data)
        return self._output.write(formatted)


# LSP: Any DataSource, Formatter, or OutputTarget can be substituted
report_csv = ReportGenerator(SalesDataSource(), CsvFormatter(), ConsoleOutput())
print(report_csv.generate())

print()

report_text = ReportGenerator(SalesDataSource(), TextFormatter(), ConsoleOutput())
print(report_text.generate())

## Summary

### The Five SOLID Principles
- **SRP**: Split classes so each has one job (User vs UserRepository vs UserService)
- **OCP**: Use ABC/Protocol to allow extension without modifying existing code
- **LSP**: Subtypes must honor the contract of their base type (avoid the Square-Rectangle trap)
- **ISP**: Use small `Protocol` classes instead of large interfaces with unused methods
- **DIP**: Inject dependencies as abstractions; let the caller decide the implementation

### Python-Specific Takeaways
- Use `Protocol` for structural typing (duck typing with type checker support)
- Use `ABC` when you need shared default implementations or want enforcement at class creation
- Prefer `Protocol` for interfaces consumed by external or third-party code
- Prefer `ABC` for internal class hierarchies where you control all subclasses
- Dependency injection in Python is simple - just pass objects to `__init__`